In [7]:
# Dependencies
import requests
import pandas as pd
from census import Census
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import numpy as np

In [9]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(api_key)

In [11]:
#Get data from the census
c = Census(api_key)

# Specify the variables 
#B08301_001E is the estimated total workers
#B08301_021E is the estimated workers who worked from home

variables = ['B08301_001E', 'B08301_021E']

# Years we are looking at
years = [2020, 2021, 2022]

# FIPS code to state name mapping
state_name = {
    '01': 'Alabama', '02': 'Alaska', '04': 'Arizona', '05': 'Arkansas', '06': 'California',
    '08': 'Colorado', '09': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia',
    '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois',
    '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana',
    '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota',
    '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada',
    '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York', '37': 'North Carolina',
    '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'Oregon', '42': 'Pennsylvania',
    '44': 'Rhode Island', '45': 'South Carolina', '46': 'South Dakota', '47': 'Tennessee', '48': 'Texas',
    '49': 'Utah', '50': 'Vermont', '51': 'Virginia', '53': 'Washington', '54': 'West Virginia',
    '55': 'Wisconsin', '56': 'Wyoming', '72': 'Puerto Rico'
}

# List to store the data for all years
all_data = []

# Loop through each year 
for year in years:
    # Get the data for all states for this year
    data = c.acs5.get(variables, geo={"for": "state:*"}, year=year)
    
    # Convert the data to a DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])
    
    # Add the year as a column to the DataFrame
    df['year'] = year
    
    # Map the state FIPS codes to state names
    df['state_name'] = df['state'].map(state_name)
    
    # Append this year's data to the all_data list
    all_data.append(df)

# Now, create the DataFrame 
df = pd.concat(all_data, ignore_index=True)

# Display the columns: State Name, Year, Total Workers, Workers Who Work From Home
df = df[['state_name', 'year', 'B08301_001E', 'B08301_021E']]

# Rename the columns for clarity
df.columns = ['State', 'Year', 'Total Workers', 'Workers Who Work From Home']

# Convert the 'Total Workers' and 'Workers Who Work From Home' columns to numeric values because strings will prevent numeric operations
df['Total Workers'] = df['Total Workers'].astype(int)
df['Workers Who Work From Home'] = df['Workers Who Work From Home'].astype(int)

# Calculate the percentage of workers who work from home
df['Percentage of Workers Who Work From Home'] = (
    df['Workers Who Work From Home'] / df['Total Workers']
) * 100

# Format the percentage column to two decimal places
df['Percentage of Workers Who Work From Home'] = df['Percentage of Workers Who Work From Home'].apply(lambda x: "{:.2f}%".format(x))

# Display the cleaned-up DataFrame
df

,State,Year,Total Workers,Workers Who Work From Home,Percentage of Workers Who Work From Home
0,California,2020,18239892,1529697,8.39%
1,West Virginia,2020,730395,33353,4.57%
2,Utah,2020,1517044,138218,9.11%
3,New York,2020,9219025,634197,6.88%
4,District of Columbia,2020,379090,46472,12.26%
...,...,...,...,...,...
148,Washington,2022,3722716,575184,15.45%
149,West Virginia,2022,723217,49655,6.87%
150,Wisconsin,2022,2974249,300409,10.10%
151,Wyoming,2022,284555,21902,7.70%


In [4]:
#Ensure the 'Percentage of Workers Who Work From Home' column is numeric
df['Percentage of Workers Who Work From Home'] = df['Percentage of Workers Who Work From Home'].apply(
    lambda x: float(x.replace('%', '').replace(',', '')) if isinstance(x, str) else x
)

# Handle NaNs in the Percentage calculation
df['Percentage of Workers Who Work From Home'] = df['Percentage of Workers Who Work From Home'].fillna(0)

#Group by 'State' and 'Year' and calculate the mean
grouped_data = df.groupby(['State', 'Year'])['Percentage of Workers Who Work From Home'].mean().reset_index()

#Pivot the data
pivoted_data = grouped_data.pivot(index='State', columns='Year', values='Percentage of Workers Who Work From Home')

#Fill any NaN values wiht 0
pivoted_data = pivoted_data.fillna("0")

# Clean up the column name
pivoted_data.columns = [f"Percentage of Workers Who Work From Home ({year})" for year in pivoted_data.columns]

#Format the values back to percentage
pivoted_data = pivoted_data.map(lambda x: f"{x:.2f}%" if isinstance(x, (float, int)) else x)

# Reset index to make 'State' a column again
pivoted_data.reset_index(inplace=True)

# Display the result
pivoted_data

,State,Percentage of Workers Who Work From Home (2020),Percentage of Workers Who Work From Home (2021),Percentage of Workers Who Work From Home (2022)
0,Alabama,4.43%,0,0
1,Alaska,6.03%,7.09%,8.06%
2,Arizona,8.88%,11.71%,14.26%
3,Arkansas,4.37%,5.63%,6.75%
4,California,8.39%,11.36%,13.63%
5,Colorado,10.80%,13.99%,16.49%
6,Connecticut,7.64%,10.40%,12.61%
7,Delaware,7.15%,9.84%,11.64%
8,District of Columbia,12.26%,19.83%,25.03%
9,Florida,7.76%,9.87%,11.94%
